Create a file called message.txt with some content of your choice. Then, write a python code in cocalc that implement the following scenario:

Prior to the following function, you need to prepare the parameters to be used for RSA encryption by the sender. These include the public and private exponents, and the integer n.

 

1) Create a sender function the does the following:

Read the content of the file message.txt.
Hash the content using SHA1 hash function.
Encrypt the hash of the message.txt using RSA encryption algorithm with the private key of the sender.
The output of this function will be both the encrypted hash value of the message using RSA, and the plaintext (original content of message.txt file).
Note here that in this scenario only message integrity is required, and no confidentiality will be provided. This is why the message is sent in plain.
 

2) Create a receiver function the does the following:

The input is the public key of the sender, the encrypted hash value, and the plaintext.
Decrypt the received hash value using the public key of the sender.
Calculate the hash value of the received plaintext.
Compare the hash value of the received plaintext with the hash value of the decrypted received hash.
 

3) perform a test that include two cases:

in first case, the message in not altered, which means that the receiver after checking will accept the received message.
In second case, the message will be modified and then fed to the receiver function. In this case, the modification will be spotted and the received message will be rejected by the receiver.


Some of the functions that were coded in the practice lecture and might be useful for this assignment.

In [15]:
from sympy import randprime
from math import gcd

def egcd(a,b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, x, y = egcd(b % a, a)
        return (g, y - (b // a) * x, x)

def modinv(b, m):
    g, x, _ = egcd(b, m)
    if g == 1:
        return x % m

In [16]:
def generatePrimes(key_size):
    prime_1 = 0
    prime_2 = 0
    while prime_1 == prime_2 or(prime_1 *prime_2) > 2**key_size:
        prime_1 = randprime(3, 2**key_size/2)
        prime_2 = randprime(3, 2**key_size/2)
    return prime_1, prime_2

In [17]:
def generate_public_Exponents(totient):
    public_exponent = 0
    for e in range(3,totient-1):
        if gcd(e,totient) == 1:
            public_exponent = e
            break
    return public_exponent

In [18]:
def generateTotient(prime_1, prime_2):
    return (prime_1 - 1) * (prime_2 - 1)

In [19]:
def rsa_encrypt(plain_text,rsa_modulus, public_exponent):
    cipher = ''.join(chr((ord(ch)**public_exponent)%rsa_modulus) for ch in plain_text)
    return cipher.encode().hex()

In [20]:
def rsa_decrypt(cipher_text, rsa_modulus, private_exponent):
    return ''.join(chr((ord(ch)**private_exponent)%rsa_modulus)
                   for ch in bytearray.fromhex(cipher_text).decode())

In [21]:
from hashlib import sha1

def hashFileContentWithSha1(fileContent):
    return sha1(fileContent.encode()).hexdigest()

In [22]:
def readFileContent(fileName):
    with open(fileName, 'r') as file:
        return file.read()

Part 1 - Sender. Sender takes fileContent hashes it, encrypts it with private key and returns encrypted hash and fileContent.

In [23]:
def senderRsa(fileName, privateKey):
    (rsa_modulus, private_exponent) = privateKey
    fileContent = readFileContent(fileName)
    hashFileContent = hashFileContentWithSha1(fileContent)
    encryptedHash = rsa_encrypt(hashFileContent, rsa_modulus, private_exponent)
    return (fileContent, encryptedHash)

Part 2 - Receiver. Receiver takes public key, encrypted hash and fileContent. It decrypts the encrypted hash with public key, hashes the fileContent and compares the two hashes. If they are equal, it returns prints "Hashes are equal", otherwise "Hashes are not equal".

In [24]:
def receiverRsa(fileContent, encryptedHash, publicKey):
    (rsa_modulus, public_exponent) = publicKey
    hashedFileContent = hashFileContentWithSha1(fileContent)
    decryptedHash = rsa_decrypt(encryptedHash, rsa_modulus, public_exponent)
    if hashedFileContent != decryptedHash:
        print("Hashes are not equal")
    else:
        print("Hashes are equal")

 Public and private keys generation for a given key size.

In [25]:
def generateKeys(key_size):
    prime_1,prime_2 = generatePrimes(key_size)
    totient = generateTotient(prime_1, prime_2)
    public_exponent = generate_public_Exponents(totient)
    private_exponent = modinv(public_exponent, totient)
    rsa_modulus = prime_1 * prime_2 # integer n from the slides
    public_key = (rsa_modulus, public_exponent)
    private_key = (rsa_modulus, private_exponent)
    return (public_key, private_key)

key_size = 10
public_key, private_key = generateKeys(key_size)

In [26]:
# part 1
(fileContent, encrypterHash) = senderRsa('message.txt', private_key)
print("File content: ", fileContent)
print("Encrypted hash: ", encrypterHash)

File content:  hello world from Budapest

Encrypted hash:  c298c899c788c982c395c395c68cc982c48dc48dc68c622a62c391c899c68cc899c391c998c68cc298c998362ac9b9c298c298c48d2ac391c298c9b9c3abc8992ac998c3abc998c391


In [27]:
# part 2
receiverRsa(fileContent, encrypterHash, public_key)

Hashes are equal


In the part 3 of the assignment, you have to alter the message and check if the receiver will see that hashes changed or not. 

In [28]:
# part 3
# not altered message
receiverRsa(fileContent, encrypterHash, public_key)

# altered message
fileContent = fileContent.replace('B', 'b')
print("Altered file content: ", fileContent)
receiverRsa(fileContent, encrypterHash, public_key)

Hashes are equal
Altered file content:  hello world from budapest

Hashes are not equal
